In [2]:
# Imports
import requests
import os

from IPython.display import HTML
from ipywidgets import interact

# Functions
def multireddit(subreddits, limit=5, orderby='score'):
    
    #If test to handle booting up
    if subreddits == "":
        return;
    
    ''' Displays an HTML listing of the top `limit` articles from the
    various `subreddits` specified by the user.  These articles will be
    sorted by the field specified by `orderby`

      - `subreddits` is a comma-separated list of subreddits (ie.
        `linux,linuxmasterrace`)

      - `orderby` is any field in the Reddit JSON data for a particular
        article (ie. `score`, `title`, `created_utc`, etc.)
    '''

    #set delimiter
    subreddits_list = subreddits.split(",")   

    # TODO: Process function arguments
    url_list = []
    article_list = []

    #Create URLs 
    for i in subreddits_list:
        new_url = 'https://www.reddit.com/r/' + i + '/.json'
        url_list.append(new_url)

    # TODO: Fetch subreddit data
    for i in url_list:
        counter = 0 

        headers  = {'user-agent': 'reddit-{}'.format(os.environ['USER'])}
        response = requests.get(i, headers=headers)

        #Try to connect to reddit to grap json
        while response.status_code != 200 and counter < 20: 
            response = requests.get(i, headers=headers)
            counter += 1 

        data = response.json()

        #Generate dictionary and populate it with json data fields
        for child in data['data']['children']:
            article_obj = {
                'title': '',
                'score': 0,
                'url': '',
                'comments': '',
                'created_utc': 0
            }

            article_obj["title"] = child['data']['title']
            article_obj['score'] = child['data']['score']
            article_obj['url'] = child['data']['url']
            article_obj['comments'] = 'https://www.reddit.com' + child['data']['permalink']
            article_obj['created_utc'] = child['data']['created_utc']

            article_list.append(article_obj)
            
    # TODO: Sort and limit data, and construct HTML results
    #Determine how to sort data
    if orderby == 'title' or orderby == 'url' or orderby == 'comments':
        article_list = sorted(article_list, key=lambda k: k[orderby])       
    else:
        article_list = sorted(article_list, key=lambda k: k[orderby], reverse=True)       
   
    #Generate HTML code    
    html = '<table><tbody>'
    for i in range(limit):

        html += '<tr>'
        html += '<td>' + str(i+1) + '. ' + '</td>'
        html += '<td>' + '<a href="' +  article_list[i]["url"] + '">' + article_list[i]["title"] + '</a></td>'
        html += '<td>' + '. ' + '</td>'
        html += '<td>' + 'Score: ' + str(article_list[i]["score"]) + '</td>'
        html += '<td>' + '. ' + '</td>'
        html += '<td><a href="' + article_list[i]["comments"] + '">Comments</a></td>'
        html += '<td>' + '. ' + '</td>'

    html += '</tbody></table>'

    display(HTML(html))
   
 #Dynamically run code
ORDERBY = {
    'Score': 'score',
    'Creation Time': 'created_utc',
    'Title': 'title'
}

interact(multireddit, subreddits='', limit=(1, 20), orderby=ORDERBY)    

A Jupyter Widget

<function __main__.multireddit>

Homework 04: Visualizing Data
===========================================

By: Ryan Karl


Activity 1: Multi-Reddit (6 Points)
---------------------------------------------


1) How you parsed and utilized the arguments to the multireddit function.

	To parse the subreddits argument, I first used a split function to seperate all of the values
	into a list.  I then used a for loop to iterate through each item in this list and concatenate the
	reddit URL (with a trailing json suffix), and then stored these URLs into another list using the append
	function.  To parse the limit and orderby arguments, I simply stored these single values in variables, for later use determining how to organize output.
	 

2) How you fetched the JSON data and iterated over the articles to extract the relevant data.
	
	To fetch the json data, I used a for loop to iterate though my list of URL values.  For each value,
	I used a while loop to repeatedly call the requests.get() method to ask for the json until ether the
	status_code = 200 or the loop executes 20 times.  I stored the json data in a variable after using the
	.json() method to access it.  To iterate over the articles, I first defined an object to hold the primary
	data attributes for each .json child entry, and used a for loop to iterate through the .json children
	and append each object to a list after it was populated with the attribute data.  I had to manually 
	go through the .json files to figure out which labels corresponded to the data I needed to extract.
	After this, I then sorted the data by calling the sorted function on the list and used an if test to
	parse the orderby variable and determine whter I needed to sort numerically or alphabetically depending 
	on what was specified (for numeric sorting I needed to set reverse=True in the sorted function so the 
	list was ordered highest to lowest).


3) How you constructed and formatted the HTML output for the relevant articles.

	To construct the HTML, I defined an html string variable, and used a for loop to loop through the range of
	the limit specified and concatenate the neccessesary headers and data to the string.  Note I needed to
	include the table and tbody headers at the beginning of the string, tr to specify tables and rows, 
	td before any data, and a href= if I wanted to embed a link.  Also note html syntax required me to add
	the corresponding closing tag at the approprate location after using each of these tags.

